In [56]:
from db.data import Data
import gurobipy as gp
from gurobipy import GRB
from scipy.special import comb

import pandas as pd

In [150]:
data = Data()

demand = data.get_demand()
waiting = data.get_waiting()
distances = data.get_distances()
crime = data.get_crime()

In [151]:
RADIUS = 5000
MAX_OFFICERS = 1
ALPHA = 1
LAMBDA = 1.35
SERVICE_TIME = 1

In [152]:
distances['in_range'] = (distances['distance'] <= RADIUS).astype(int)

In [153]:
distances = distances.pivot(index='WPname',columns='DAuid',values='in_range')

In [154]:
distances = distances.loc[:, (distances.sum() > 0)]
distances

DAuid,59151838,59151839,59151840,59151841,59151842,59151843,59151844,59151845,59151846,59151847,...,59154008,59154009,59154010,59154011,59154020,59154021,59154022,59154023,59154024,59154025
WPname,,,,,,,,,,,,,,,,,,,,,
APH Matthew Park,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Aria Banquet Convention Center,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bear Creek Park,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Big plaza at 88 and scott road,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Blumsen Park,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Shops at Morgan Crossing Outlet,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,1,1,1,1
Tynehead Regional Park,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,1,1,1
Unwin Park,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [155]:
waiting_points = list(distances.columns)
demand = demand[demand['DAuid'].isin(waiting_points)]

In [156]:
distances = distances.drop('Holland Park').rename(index={"Big plaza at 88 and scott road": "Plaza at 88 and scott road"})
coverage = {}
for index, row in distances.iterrows():
    coverage[index] = [set(row[row==1].keys())]

In [157]:
pop_dict = pd.Series(demand.population_val.values,index=(demand.DAuid)).to_dict()

In [158]:
MAX_OFFICERS=5
ALPHA=1
regions, population = gp.multidict(pop_dict)
waits, covered = gp.multidict(coverage)

prob = [comb(MAX_OFFICERS,k)*((0.6)**k)*((0.4)**(MAX_OFFICERS-k)) for k in range(0,MAX_OFFICERS+1)]

In [159]:
m = gp.Model("SPMARP")
m.Params.LogToConsole = 0 #dont print model information

In [160]:
wait_size = [(name,size) for name in list(waiting['name']) for size in range(0,MAX_OFFICERS+1)]
demand_size = [(DAuid,size) for DAuid in list(distances.columns) for size in range(0,MAX_OFFICERS+1)]

In [161]:
#variable about where to place officers
build = m.addVars(wait_size, vtype=GRB.BINARY, name="wait_points")
is_covered = m.addVars(demand_size, vtype=GRB.BINARY, name="Is_covered")
build_change = m.addVars(wait_size, vtype=GRB.BINARY, name="wait_change")

In [162]:
#sum demand points only where officers can reach
for size in range(0,MAX_OFFICERS+1):
    m.addConstrs((gp.quicksum(build[wait,size] for wait in waits if r in covered[wait]) >= is_covered[r,size]
                            for r in regions), name="Build2cover")

In [163]:
#only allow max officers
for size in range(0,MAX_OFFICERS+1):
    m.addConstr(gp.quicksum(build[wait,size] for wait in waits) == size, name="officers")

In [164]:
#Control waiting site change
for size in range(1,MAX_OFFICERS):
    m.addConstrs( (build[wait,size] - build[wait,size+1] <= build_change[wait,size] for wait in waits), name="cease_wait")

In [165]:
#Max amount of location changes
for size in range(1,MAX_OFFICERS):
    m.addConstrs( (build_change[wait,size] <= ALPHA for wait in waits), name="cease_wait")

In [166]:
obj = gp.quicksum(prob[k]*population[r]*is_covered[r,k] for r in regions for k in range(1,MAX_OFFICERS+1))

In [167]:
m.setObjective(obj, GRB.MAXIMIZE)

In [168]:
m.optimize()

GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [ ]:
for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        print(f"\n Place officer at location {tower}.")

In [ ]:
#STATIC CASE
MAX_OFFICERS=2
m = gp.Model("SPMARP")
m.Params.LogToConsole = 0 #dont print model information
#variable about where to place officers
build = m.addVars(list(waiting['name']), vtype=GRB.BINARY, name="wait_points")
is_covered = m.addVars(list(distances.columns), vtype=GRB.BINARY, name="Is_covered")

#sum demand points only where officers placed
m.addConstrs((gp.quicksum(build[t] for t in waits if r in covered[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")

#only allow max officers
m.addConstr(gp.quicksum(build[t] for t in waits) == MAX_OFFICERS, name="officers")

m.setObjective(gp.quicksum(is_covered[r]*population[r] for r in regions), GRB.MAXIMIZE)

m.optimize()

print(f'model size={MAX_OFFICERS}')
for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        print(f"\n Place officer at location {tower}.")

total_population = 0

for region in regions:
    total_population += population[region]

percent = round(100*m.objVal/total_population, 2)

print(f"\n The population coverage associated to the patrol officer distribution is: {percent} %\n\n")   

In [ ]:
build['Bear Creek Park'].getAttr('x') #value of variable, 1 means place officer